<a href="https://colab.research.google.com/github/HappyJJins/KakaoArena/blob/master/Song_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys
from google.colab import drive, auth

ROOT = '/content/drive'
drive.mount(ROOT)

my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)

In [7]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from scipy import stats

In [8]:
val = pd.read_json('/content/drive/My Drive/Colab Notebooks/카카오X멜론 공모전/data/val.json', typ='frame')
song_meta = pd.read_json('/content/drive/My Drive/Colab Notebooks/카카오X멜론 공모전/data/song_meta.json', typ = 'frame')
train = pd.read_json('/content/drive/My Drive/Colab Notebooks/카카오X멜론 공모전/data/train.json', typ = 'frame')

In [67]:
val[10:20]

,tags,id,plylst_title,songs,like_cnt,updt_date
10,[],126307,,"[362319, 38935, 119190, 452702]",272,2011-07-28 14:44:58.000
11,"[아침, 눈, 크리스마스]",136650,,"[174132, 257367, 539802, 568691, 296891, 38471...",6,2017-11-24 11:26:05.000
12,[EDM],53131,,"[392257, 668879, 460746, 680240, 490966, 25010...",268,2016-09-19 15:40:31.000
13,[],44037,,"[20294, 183236, 444525, 509867, 650133, 110587...",4,2018-01-08 10:36:19.000
14,[],64105,,"[131609, 327419, 454168, 702488, 324003, 47234...",45,2013-01-08 23:08:40.000
15,[],40637,,"[354004, 199262, 637107, 452514, 310090, 13281...",2,2017-09-26 11:20:56.000
16,[],41828,,"[648204, 8297, 278249]",32,2013-02-28 11:47:49.000
17,[생각나],2380,다시 생각나는 그 사람,[],6,2016-03-19 23:53:20.000
18,[],87846,,"[564439, 16513, 12841, 395460, 449327, 113160]",68,2019-06-02 11:32:17.000
19,[],94446,,"[482423, 543094, 131295, 198666, 61771, 272180...",1,2020-04-10 13:57:54.000


In [9]:
# 플레이리스트 아이디(id)와 수록곡(songs) 추출
plylst_song_map = train[['id', 'songs']]

# unnest songs
plylst_song_map_unnest = np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))), 
        np.concatenate(plylst_song_map.songs.values)
    )
)

# unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map = pd.DataFrame(data = plylst_song_map_unnest[0], columns = plylst_song_map.columns)
plylst_song_map['id'] = plylst_song_map['id'].astype(str)
plylst_song_map['songs'] = plylst_song_map['songs'].astype(str)

# unnest 객체 제거
del plylst_song_map_unnest

In [10]:
# 플레이리스트 아이디(id)와 매핑된 태그(tags) 추출
plylst_tag_map = train[['id', 'tags']]

# unnest tags
plylst_tag_map_unnest = np.dstack(
    (
        np.repeat(plylst_tag_map.id.values, list(map(len, plylst_tag_map.tags))), 
        np.concatenate(plylst_tag_map.tags.values)
    )
)

# unnested 데이터프레임 생성 : plylst_tag_map
plylst_tag_map = pd.DataFrame(data = plylst_tag_map_unnest[0], columns = plylst_tag_map.columns)
plylst_tag_map['id'] = plylst_tag_map['id'].astype(str)

# unnest 객체 제거
del plylst_tag_map_unnest

In [11]:
# 곡 별 매핑 되는 장르 수?

# 곡 아이디(id)와 대분류 장르코드 리스트(song_gn_gnr_basket) 추출
song_gnr_map = song_meta.loc[:, ['id', 'song_gn_gnr_basket']]

# unnest song_gn_gnr_basket
song_gnr_map_unnest = np.dstack(
    (
        np.repeat(song_gnr_map.id.values, list(map(len, song_gnr_map.song_gn_gnr_basket))), 
        np.concatenate(song_gnr_map.song_gn_gnr_basket.values)
    )
)

# unnested 데이터프레임 생성 : song_gnr_map
song_gnr_map = pd.DataFrame(data = song_gnr_map_unnest[0], columns = song_gnr_map.columns)
song_gnr_map['id'] = song_gnr_map['id'].astype(str)
song_gnr_map.rename(columns = {'id' : 'song_id', 'song_gn_gnr_basket' : 'gnr_code'}, inplace = True)

# unnest 객체 제거
del song_gnr_map_unnest

In [12]:
# 1. 플레이리스트-곡 테이블 plylst_song_map + 플레이리스트-태그 테이블 plylst_tag_map join 테이블 생성 : plylst_song_tag_map
plylst_song_tag_map = pd.merge(plylst_song_map, plylst_tag_map, how = 'left', on = 'id')

# # 2. 1번 테이블 plylst_song_tag_map + 곡 장르 테이블 song_gnr_map join
plylst_song_tag_map = pd.merge(plylst_song_tag_map, song_gnr_map, how = 'left', left_on = 'songs', right_on = 'song_id')
plylst_song_tag_map

,id,songs,tags,song_id,gnr_code
0,61281,525514,락,525514,GN1400
1,61281,129701,락,129701,GN0900
2,61281,129701,락,129701,GN1000
3,61281,383374,락,383374,GN1000
4,61281,562083,락,562083,GN0900
...,...,...,...,...,...
29620344,100389,13759,팝송모음,13759,GN1000
29620345,100389,154078,노래추천,154078,GN1000
29620346,100389,154078,팝송추천,154078,GN1000
29620347,100389,154078,팝송,154078,GN1000


In [13]:
# 곡 아이디(id)와 대분류 장르코드 리스트(song_gn_gnr_basket) 추출
song_gnr_map = song_meta.loc[:, ['id', 'song_gn_gnr_basket']]

# unnest song_gn_gnr_basket
song_gnr_map_unnest = np.dstack(
    (
        np.repeat(song_gnr_map.id.values, list(map(len, song_gnr_map.song_gn_gnr_basket))), 
        np.concatenate(song_gnr_map.song_gn_gnr_basket.values)
    )
)

# unnested 데이터프레임 생성 : song_gnr_map
song_gnr_map = pd.DataFrame(data = song_gnr_map_unnest[0], columns = song_gnr_map.columns)
song_gnr_map['id'] = song_gnr_map['id'].astype(str)
song_gnr_map.rename(columns = {'id' : 'song_id', 'song_gn_gnr_basket' : 'gnr_code'}, inplace = True)

# unnest 객체 제거
del song_gnr_map_unnest

In [14]:
# 태그별 매핑된 장르 리스트
tag_gnr = pd.DataFrame()
tag_gnr['gnr_code'] = plylst_song_tag_map.groupby('tags').gnr_code.unique()
tag_gnr['gnr_code'] = tag_gnr['gnr_code'].apply(lambda x: x.tolist())
tag_gnr.reset_index(level=['tags'], inplace = True)

In [15]:
val_train = val
# song id와 playlist id type을 str으로
val_train['songs'] = val_train['songs'].apply(lambda x: list(map(str, x)))
val_train['id'] = val_train['id'].astype(str)

In [16]:
def test_algo(id):
  # 입력된 playlist id에 수록된 tag와 song list
  tags = val_train[val_train['id'] == id]['tags'].tolist()[0]
  songs = val_train[val_train['id'] == id]['songs'].tolist()[0]
  # 앞의 tag와 song에 해당하는 장르 list
  tags_gnr = list(map(lambda x: tag_gnr[tag_gnr['tags'] == x]['gnr_code'].tolist()[0], tags))
  songs_gnr = list(map(lambda x: song_gnr_map[song_gnr_map['song_id'] == x]['gnr_code'].tolist()[0], songs))
  songs_gnr.append(tags_gnr)
  
  key = re.compile('[G|N|0-9]+')
  test = key.findall(str(songs_gnr)) #nan값 제거

  gnr = pd.Series(test)

  return gnr.value_counts().head(5) # 많이 매핑된 장르 5개

### Plylst 번호 입력

In [17]:
plylst_id = '136650'
val_train[val_train['id'] == plylst_id]

,tags,id,plylst_title,songs,like_cnt,updt_date
11,"[아침, 눈, 크리스마스]",136650,,"[174132, 257367, 539802, 568691, 296891, 38471...",6,2017-11-24 11:26:05.000


In [18]:
test_algo(plylst_id) # '136650'번 playlist에 매핑된 장르 상위 5개

GN1700    10
GN1400     5
GN0900     5
GN1100     4
GN0800     4
dtype: int64

In [19]:
test = test_algo(plylst_id)
test_idx = test.index.tolist()
test_idx

['GN1700', 'GN1400', 'GN0900', 'GN1100', 'GN0800']

In [27]:
def cosine(gnr_code) :
    cosine_sim = np.load('/content/drive/My Drive/Colab Notebooks/카카오X멜론 공모전/data/gn_cos/{}.npy'.format(gnr_code))
    return cosine_sim

In [24]:
# 발라드 장르 코사인 유사도 매트릭스
# 용량이 커서 그런가 안 돌아감. 
cosine('GN0400')

OSError: ignored

In [29]:
cosine_sim = cosine(test_idx[0]) # 가장 많이 매핑된 장르인 GN1700의 코사인 유사도 매트릭스
cosine_sim

PermissionError: ignored

In [71]:
test = pd.read_json('/content/drive/My Drive/Colab Notebooks/카카오X멜론 공모전/data/gnr_description/{}.json'.format(test_idx[0]), orient = 'table')
test

,song_id,gnr_code,description
0,5,GN1700,"[['2010', '자라섬', '재즈', '페스티벌', '국내', '아티스트', '..."
1,12,GN1700,"[['요즘', '가을밤에', '딱', '어울리는', '빅밴드', '재즈', '재즈'..."
2,38,GN1700,"[['띵곡모여라', '장르', '상관', '없는', '명곡', '모음', '띵곡여기..."
3,41,GN1700,"[['winter', 'jazz', '재즈'], ['클스', '재즈', '겨울', ..."
4,73,GN1700,"[['Gypsy', 'Jazz', 'Collection', '재즈']]"
...,...,...,...
41503,707811,GN1700,"[['재즈', '어렵지', '않아요', '입문용', '음반', 'part', '2'..."
41504,707846,GN1700,"[['멍때리기', '좋은', '느낌있는', '재즈음악', '휴식', '힐링', '기..."
41505,707958,GN1700,"[['Tigran', 'Hamasyan', '테크닉과', '과감함을', '지닌', ..."
41506,707979,GN1700,"[['재즈의', '변절자', '혹은', '재즈', '트렌드의', '선구자', 'He..."


In [73]:
indices = pd.Series(test.index, index = test['song_id'])

In [72]:
def get_recommendations(song_id, num) :
    idx = indices[song_id]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num]
    song_indices = [i[0] for i in sim_scores]
    return test.iloc[song_indices][['song_id']]

In [74]:
# GN1700에 속한 '707983' 곡과 유사한 상위 30개 곡리스트
get_recommendations('707983',30)

,song_id
3951,67225
10478,179602
15920,270117
16341,276701
19832,335052
33349,568193
41507,707983
7363,125888
309,5260
2017,34282


In [76]:
def gnr_song(plylst_id):
    songs = val_train[val_train['id'] == plylst_id]['songs'].tolist()[0] # 해당 playlist에 수록된 곡
    test_song = test['song_id'] # 해당 playlist에서 가장 많이 매핑된 장르에 속한 모든 곡
    test_song = test_song.values.tolist()
    return list(set(test_song) & set(songs)) # 해당 playlist에서 가장 많이 매핑된 장르에 속하는 곡

In [79]:
test_song = gnr_song(plylst_id) # '136650'번 playlist에서 가장 많이 매핑된 장르(GN1700)에 속하는 곡
test_song

['594472', '568691', '384718', '539802', '296891', '367173']

In [1]:
# test_song 리스트의 곡마다 유사곡 상위 100개씩 추출
recommend = pd.DataFrame()
pd.Series(list(map(lambda x:get_recommendations(x, 100).song_id.values.tolist() ,test_song)))

# for i in test_song:
#     recommend = recommend.append(get_recommendations(i,100))

NameError: ignored

In [82]:
recommend = recommend.reset_index()
recommend

,index,song_id
0,7179,122505
1,12550,213930
2,17554,296891
3,18347,310906
4,22684,384718
...,...,...
589,19686,332750
590,18551,314183
591,25180,427980
592,14041,238808


In [84]:
recommend_cnt = recommend.groupby('song_id').count().rename({'index':'count'})
recommend_cnt

,index
song_id,
101653,5
105090,1
109858,1
110797,6
112838,6
...,...
76173,6
7908,5
80228,5


In [ ]:
recommend_cnt = recommend_cnt.sort_values('index',ascending=False)

In [ ]:
result_song = recommend_cnt.head(150)

In [ ]:
result_song = result_song.reset_index()

In [ ]:
result_song

,song_id,index
0,567438,4
1,213364,4
2,4706,4
3,608505,4
4,135506,3
...,...,...
145,563453,1
146,561695,1
147,562041,1
148,526953,1


In [ ]:
result_songs = result_song['song_id'].tolist()

In [ ]:
result_songs

['567438',
 '213364',
 '4706',
 '608505',
 '135506',
 '258646',
 '72002',
 '543606',
 '381874',
 '541434',
 '320827',
 '274504',
 '677277',
 '419278',
 '159366',
 '177414',
 '427021',
 '213289',
 '168031',
 '457552',
 '348089',
 '481160',
 '284111',
 '2748',
 '58704',
 '397222',
 '20403',
 '203660',
 '411150',
 '615235',
 '678161',
 '191782',
 '286829',
 '191620',
 '518636',
 '517472',
 '298949',
 '681346',
 '333595',
 '176435',
 '175720',
 '591699',
 '616347',
 '227306',
 '619381',
 '223540',
 '268645',
 '2666',
 '567766',
 '630941',
 '62174',
 '27885',
 '642487',
 '376435',
 '243719',
 '643643',
 '173533',
 '551757',
 '548304',
 '381331',
 '22907',
 '542315',
 '623145',
 '555624',
 '99193',
 '74920',
 '696973',
 '146989',
 '146668',
 '703985',
 '502335',
 '441558',
 '444194',
 '137421',
 '44782',
 '510395',
 '44842',
 '8153',
 '452185',
 '458311',
 '106500',
 '477206',
 '699953',
 '66416',
 '696317',
 '166076',
 '507711',
 '485748',
 '481694',
 '482563',
 '579604',
 '577877',
 '48396

In [ ]:
result = list(set(result_songs)-set(test_pass))[0:100]

In [ ]:
len(result)

100

In [ ]:
result

['483962',
 '696973',
 '567766',
 '588750',
 '58704',
 '542315',
 '681346',
 '587390',
 '175720',
 '484934',
 '480533',
 '223540',
 '476132',
 '540634',
 '284111',
 '2748',
 '488425',
 '62174',
 '561695',
 '166076',
 '566472',
 '381874',
 '696317',
 '479747',
 '20403',
 '58655',
 '541434',
 '159366',
 '623145',
 '427021',
 '59441',
 '56981',
 '227306',
 '588885',
 '563453',
 '526953',
 '583327',
 '66416',
 '444194',
 '591973',
 '27885',
 '411150',
 '243719',
 '589871',
 '564936',
 '457552',
 '59290',
 '608505',
 '581080',
 '481160',
 '44782',
 '58828',
 '191620',
 '504906',
 '565696',
 '397222',
 '551757',
 '594784',
 '452185',
 '348089',
 '642487',
 '48093',
 '191782',
 '508719',
 '548304',
 '419278',
 '699953',
 '482563',
 '173533',
 '286829',
 '381331',
 '274504',
 '515625',
 '587188',
 '510395',
 '678161',
 '146668',
 '176435',
 '106500',
 '213364',
 '562041',
 '44842',
 '488946',
 '492874',
 '572456',
 '571016',
 '643643',
 '518636',
 '484991',
 '135506',
 '490746',
 '213289',
 '6